In [1]:
import threading
import urllib.request
import os
import queue
import time

# Thread-safe queues to hold download and processing tasks
download_queue = queue.Queue()
process_queue = queue.Queue()
# List to store successfully downloaded files
downloaded_files = []
processed_files = []

def download_file(url, filename):
    """Downloads a file from a given URL and saves it to the specified filename."""
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    print(f"\nDownloading {filename} from {url}...")
    try:
        urllib.request.urlretrieve(url, filename)
        print(f"\n{filename} downloaded successfully.")
        # Append to downloaded files
        downloaded_files.append(filename)
        # Add the downloaded file to the processing queue
        process_queue.put(filename)
    except Exception as e:
        print(f"Error downloading {filename}: {e}")

def process_file(filename):
    """Processes the downloaded file (e.g., converts format, resizes image, etc.)."""
    print(f"\nProcessing {filename}...")
    time.sleep(1)  # Simulate processing time
    processed_files.append(filename)
    print(f"\n{filename} processed successfully.")

def worker_download():
    """Thread worker function to process download tasks."""
    while True:
        url, filename = download_queue.get()
        if url is None:  
            break
        download_file(url, filename)
        download_queue.task_done()

def worker_process():
    """Thread worker function to process files."""
    while True:
        filename = process_queue.get()
        if filename is None:  
            break
        process_file(filename)
        process_queue.task_done()

def multi_threaded_download(files_to_download):
    """Starts multiple threads to download files."""
    threads = []
    for _ in range(4): 
        thread = threading.Thread(target=worker_download)
        thread.start()
        threads.append(thread)

    for file_info in files_to_download:
        download_queue.put((file_info["url"], file_info["filename"]))

    download_queue.join()

    for _ in range(len(threads)):
        download_queue.put((None, None))  # Stop signal

    for thread in threads:
        thread.join()

def multi_threaded_process():
    """Starts multiple threads to process files."""
    threads = []
    for _ in range(4): 
        thread = threading.Thread(target=worker_process)
        thread.start()
        threads.append(thread)

    process_queue.join()

    for _ in range(len(threads)):
        process_queue.put(None)  # Stop signal

    for thread in threads:
        thread.join()

def single_threaded_download(files_to_download):
    """Downloads files sequentially."""
    for file_info in files_to_download:
        download_file(file_info["url"], file_info["filename"])

def measure_performance(files_to_download):
    """Measures and compares the performance of single vs multi-threaded downloads and processing."""
    
    # Measure single-threaded download time
    start_time = time.time()
    single_threaded_download(files_to_download)
    single_threaded_duration = time.time() - start_time
    print(f"\nSingle-threaded download completed in {single_threaded_duration:.2f} seconds.")

    # Measure multi-threaded download time
    start_time = time.time()
    multi_threaded_download(files_to_download)
    multi_threaded_duration = time.time() - start_time
    print(f"\nMulti-threaded download completed in {multi_threaded_duration:.2f} seconds.")

    # Process files using multi-threading
    multi_threaded_process()
    print(f"\nProcessed files: {processed_files}")

def main():
    # List of files to download (with filenames)
    files_to_download = [
        {"url": "https://www.learningaboutelectronics.com/images/LED.png", "filename": r"C:\downloads\file1.png"},
        {"url": "https://www.learningaboutelectronics.com/images/Resistor.png", "filename": r"C:\downloads\file3.png"},
        {"url": "https://www.learningaboutelectronics.com/images/Diode.png", "filename": r"C:\downloads\file4.png"},
        # You can add more URLs here to test performance
    ]

    # Measure performance
    measure_performance(files_to_download)

    print("\nDownloaded files:")
    for file in downloaded_files:
        print(file)

if __name__ == "__main__":
    main()




C:\downloads\file1.png downloaded successfully.


C:\downloads\file3.png downloaded successfully.


C:\downloads\file4.png downloaded successfully.

Single-threaded download completed in 4.61 seconds.




C:\downloads\file1.png downloaded successfully.
C:\downloads\file4.png downloaded successfully.


C:\downloads\file3.png downloaded successfully.

Multi-threaded download completed in 1.23 seconds.

Processing C:\downloads\file1.png...

Processing C:\downloads\file3.png...

Processing C:\downloads\file4.png...

Processing C:\downloads\file4.png...

C:\downloads\file1.png processed successfully.

Processing C:\downloads\file1.png...

C:\downloads\file4.png processed successfully.

Processing C:\downloads\file3.png...

C:\downloads\file4.png processed successfully.

C:\downloads\file3.png processed successfully.

C:\downloads\file1.png processed successfully.

C:\downloads\file3.png processed successfully.

Processed files: ['C:\\downloads\\file1.png', 'C:\\downloads\\file4.png', 'C:\